<a href="https://colab.research.google.com/github/ayushi15092002/mental-health-workload/blob/main/Feature_Selection_Wrapper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

In [ ]:
pip install mlxtend --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.6 MB/s eta 0:00:00
  Attempting uninstall: mlxtend
    Found existing installation: mlxtend 0.14.0
    Uninstalling mlxtend-0.14.0:
      Successfully uninstalled mlxtend-0.14.0


In [ ]:
import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
 
# plt.style.use('seaborn-colorblind')
# %matplotlib inline
# from feature_selection import filter_method as ft

## Load Dataset

In [ ]:
data = pd.read_excel(r'/content/drive/My Drive/drdo/features/min_max_norm.xlsx')

In [ ]:
data.head(5)

,Coeffiecient_of_Variation,Mean_of_Vertex_to_Vertex_Slope,Variance_of_Vertex_to_Vertex_Slope,Hjorth_Activity,Hjorth_Mobility,Hjorth_Complexity,Kurtosis,2nd_Difference_Mean,2nd_Difference_Max,Skewness,...,28_Wavelet_Approximate_Std_Deviation,28_Wavelet_Approximate_Energy,28_Wavelet_Detailed_Mean,28_Wavelet_Detailed_Std _Deviation,28_Wavelet_Detailed_Energy,28_Wavelet_Approximate_Entropy,28_Wavelet_Detailed_Entropy,Hurst_Exponent,Permutation_Entropy,y
0,0.534439,0.614380,0.096161,0.020099,0.555111,0.052773,0.159801,0.248315,0.320428,0.528456,...,0.103340,0.009270,0.384685,0.344930,0.171401,0.184330,0.011559,0.723232,0.818775,1.0
1,0.534315,0.649929,0.068544,0.025495,0.455425,0.084819,0.133192,0.202098,0.254608,0.543443,...,0.146594,0.008348,0.571264,0.323252,0.155246,0.166751,0.010475,0.728989,0.812935,1.0
2,0.534344,0.583582,0.112827,0.027777,0.494449,0.070979,0.099703,0.281429,0.347117,0.595850,...,0.081346,0.004189,0.484646,0.452760,0.262473,0.278811,0.005396,0.751985,0.878779,1.0
3,0.542825,0.572106,0.097661,0.033724,0.452482,0.085299,0.141227,0.260114,0.302845,0.450263,...,0.141749,0.008017,0.776456,0.437634,0.248645,0.263571,0.010023,0.590368,0.849525,1.0
4,0.534477,0.629300,0.097666,0.029060,0.475090,0.078828,0.121195,0.253848,0.293299,0.568707,...,0.119258,0.008177,0.550035,0.382984,0.201500,0.215768,0.010123,0.612329,0.855054,1.0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(labels=['y'], axis=1), 
                                                    data.y, test_size=0.2,
                                                    random_state=0)
X_train.shape, X_test.shape

((9256, 246), (2314, 246))

## Forward Selection
 

In [ ]:
# step forward feature selection
# select top 10 features based on the optimal roc_auc and RandomForest Classifier

sfs1 = SFS(RandomForestClassifier(n_jobs=-1,n_estimators=5), 
           k_features=10, 
           forward=True, 
           floating=False, 
           verbose=1,
           scoring='roc_auc',
           cv=3)

sfs1 = sfs1.fit(np.array(X_train), y_train)

Streaming output truncated to the last 5000 lines.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 349, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partit

In [ ]:
selected_feat1= X_train.columns[list(sfs1.k_feature_idx_)]
selected_feat1

Index(['Coeffiecient_of_Variation'], dtype='object')

## Backward Elimination

In [ ]:
# step backward feature selection
# select top 10 features based on the optimal roc_auc and RandomForest Classifier

sfs2 = SFS(RandomForestClassifier(n_jobs=-1,n_estimators=5), 
           k_features=10, 
           forward=False, 
           floating=False, 
           verbose=1,
           scoring='roc_auc',
           cv=3)

sfs2 = sfs1.fit(np.array(X_train.fillna(0)), y_train)

Streaming output truncated to the last 5000 lines.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 349, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partit

In [ ]:
selected_feat2= X_train.columns[list(sfs2.k_feature_idx_)]
selected_feat2


Index(['Coeffiecient_of_Variation'], dtype='object')

Note that SFS and SBE return different results

[link text](https://)## Exhaustive Feature Selection

In [ ]:
efs1 = EFS(RandomForestClassifier(n_jobs=-1,n_estimators=5, random_state=0), 
           min_features=1,
           max_features=6, 
           scoring='roc_auc',
           print_progress=True,
           cv=2)

# in order to shorter search time for the demonstration
# we only try all possible 1,2,3,4,5,6
# feature combinations from a dataset of 10 features
# np.array(X_train[X_train.columns[0:10]].fillna(0))
efs1 = efs1.fit(X_train, y_train)

Streaming output truncated to the last 5000 lines.
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 349, in _score
    raise ValueError("{0} format is not supported".format(y_type))
ValueError: multiclass format is not supported

  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 103, in __call__
    score = scorer._score(cached_call, estimator, *args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_scorer.py", line 349, in _score
 

In [ ]:
selected_feat3= X_train.columns[list(efs1.best_idx_)]
selected_feat3